In [8]:
from keras.datasets import mnist
from keras.layers import Input,Dense,Reshape,Flatten,BatchNormalization,LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# Defining image dimensions

imgRows = 28
imgCols = 28
channels = 1
imgShape = (imgRows,imgCols,channels)

In [10]:
def generator():

    """
        Creates a generator model for a Generative Adversaial Network (GAN).

        The generator takes a noise vector as input and transforms it into realistic looking image through a series of fully connected layers,Leaky Relu 
        activation , batch normalizations and reshaping. The final output is scaled tot the range [-1,1] using the 'tanh' activation function,
        making it suitable for image generation tasks.
    
        return:
            keras.Model : A keras model that maps noise vectors to generated images.
    
    """

    #Define the shape of the noise vector ; this will serve as the input to the generastor 
    # typically used in GANS, the noise vector allows the model to generate diverse outputs
    noise_shape = (100,)

    # Initilize a sequential model , which is a linear stack of layers
    model = Sequential()

    #Add a dense layer with 256 neurons, this layer acts as the first fully connected layer, transforming the 
    #input noise vector into a higher dimensional feature space.
    #the input_shape defines the expected input noise shape dimensions 
    model.add(Dense(256,input_shape = noise_shape))
    
    # Add a LekyRelu activation function with a small negative slope defined by 'alpha'
    # LeakyRelu is a variant of the standard Relu (Rectified linear unit) actibvtion function
    # While standard RELU sets the value to 0 for all the negative inputs , this allows a small, non zero gradient for negative inputs
    # This helps mitigate the "dying RELU" problem, where neurons become inactive and stop learning due to 0 gradient
    # the alpha parameter defines the slope of the activatioin function for negative inputs. A smaller alpha means means less contibution from negative values 
    # large alpha means it allows more contribution from neagtive values.
    # this activation helps prevent the 'dying relu' problem by allowing small gradient for negative inputs
    model.add(LeakyRelu(alpha = 0.2))

    # Add batch normalization layer to stablize and accelarate training by normalizing the activations of the previous layer.
    # The momentum parameter controls how much of the past running statics to use.
    # This layer also prevents internal covariate shift and improves the models generalization ability
    model.add(BatchNormalization(momentum = 0.8))

    # Add a Dense layer with 512 neurons to further expand the feature space.
    model.add(Dense(512))
    model.add(LeakuRelu(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add another Dense layer with 1024 neurons for further expansion.
    model.add(Dense(1024))
    model.add(LeakyRelu(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add the output Dense Layer to produce the final generated image.
    # np.prod(image shape) caluclates the total number of pixels (flattned shape) of the output image.
    # the activation function 'tanh' ensures that the output values are scaled bwteen -1 and 1 which is common for image generation tasks 
    model.add(Dense(np.prod(imgShape),activation = 'tanh'))

    # Reshape the output to match the desired image dimensions(img Shape).
    model.add(Reshape(imgShape))

    #print model summary of the model architecture.
    model.summary()

    # define the input to the generastor which is the noise vector
    noise = Input(shape = noise_shape)

    # pass the noise vector through the model to generate image 
    img = model(noise)

    # return the generator model which maps noise vector to generated images
    return Model(noise, img)

In [11]:
def discriminator():

    model = Sequntial()
    model.add(Flatten(inputShape = imgShape))
    model.add(Dense(512))
    model.add(LeakyRelu(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyRelu(alpha=0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    model.summary()
    img = Input(shape = imgShape)
    validity = model(img)
    return Model(img,validity)
    
    

In [12]:
 (X_train, _), (_, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [14]:
X_train.dtype

dtype('uint8')